In [ ]:
import time, sys, os, copy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy as np
import pandas as pd
from math import sqrt
import random
import subprocess
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.lines as lines
import matplotlib.patches as patches

from multiprocessing import Pool
import multiprocessing

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, TimeDistributed, BatchNormalization
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import RMSprop, Adagrad, Adam, SGD
#from keras.models import load_model
import tensorflow.keras.backend as K



import warnings
warnings.filterwarnings("ignore")

weeks = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if 'week5' in filename:
            weeks = os.path.join(dirname, filename)
        if 'plays' in filename: 
            plays = os.path.join(dirname, filename)

# Description 
In this notebook, I try to predict players next position, based on previous positions, velocity and distance to the ball respctively. To do so, I used a sample dataset of *gameId== 2018100400 and playId==51* to overfit Models. As can be seen in this notebook the model is basically collapsing to the mean. So here is my question: 
### Why is this happening? 
1. I tested it with LSTMs and Conv Networks
2. Should be the Datapreparation


### Solution
1. Conv Network: I didnt really care to get it to work
2. LSTM Network:*Even tho I thought for Overfitting the data, It doesnt matter if I normalize the data, It converges if substracting the mean.*

## Loading the example data

In [ ]:
datadf =  pd.read_csv(weeks)
datadf.head()

In [ ]:
def compute_speed(data):
    data['v_x'] = np.sin(data['dir']*np.pi/180) * data['s']
    data['v_y'] = np.cos(data['dir']*np.pi/180) * data['s']
    data['v_theta'] = np.arctan(data['v_x']/data['v_y']) if data['v_y'] != 0 else 0 
    return data


In [ ]:
player_trajectories = []
play = datadf.query('gameId== 2018100400 and playId==51')
play = play.apply(compute_speed, axis=1)
home = play.query('team=="home"')
away = play.query('team=="away"')
football = play.query('team=="football"')
football = football.sort_values('frameId')
names = play['displayName'].value_counts().index


for player in away['displayName'].value_counts().index:
    trajectory = play.loc[play['displayName']==player]
    player_seq = trajectory.sort_values(['frameId'])
    dist_to_ball_x = np.absolute(player_seq[['x']].to_numpy()-football[['x']].to_numpy())
    dist_to_ball_y = np.absolute(player_seq[['y']].to_numpy()-football[['y']].to_numpy())
    player_seq = player_seq[['x','y','v_x','v_y']].to_numpy()
    #final_player_traj = np.hstack((player_seq, dist_to_ball_x, dist_to_ball_y))
    player_trajectories.append(player_seq)

for player in home['displayName'].value_counts().index:
    trajectory = play.loc[play['displayName']==player]
    player_seq = trajectory.sort_values(['frameId'])
    dist_to_ball_x = np.absolute(player_seq[['x']].to_numpy()-football[['x']].to_numpy())
    dist_to_ball_y = np.absolute(player_seq[['y']].to_numpy()-football[['y']].to_numpy())
    player_seq = player_seq[['x','y','v_x','v_y']].to_numpy()
    #final_player_traj = np.hstack((player_seq, dist_to_ball_x, dist_to_ball_y))
    player_trajectories.append(player_seq)



football_seq = football[['x','y']].to_numpy()

a = np.hstack(player_trajectories)
a = np.hstack((a,football_seq))
#a = a[10:]- np.mean(a, axis=0)
a = np.expand_dims(a[10:], axis=0)

## Return sequences with a simple rolling window approach

In [ ]:
def get_sequences(single_plays, policies, sequence_length, overlap, n_fts =6):
    training_complete = []
    target_complete = []
    for pol in range(policies): 
        train = []
        target = []
        for i in single_plays:
            for j in range(0, i.shape[0]+sequence_length, sequence_length-overlap):
                try:
                    if  j>sequence_length-1:
                        targets = i[j+1, pol*n_fts:pol*n_fts+2]
                        training = i[j-sequence_length:j]    
                        train.append(np.expand_dims(training,0))
                        target.append(np.expand_dims(targets,0))
                    else:
                        targets = i[j+1, pol*n_fts:pol*n_fts+2]
                        training = i[:j]
                        training = np.vstack((np.tile(i[0], (sequence_length-j,1)), training))
                        train.append(np.expand_dims(training,0))
                        target.append(np.expand_dims(targets,0))
                except:
                    pass
        training_complete.append(np.vstack(train))
        target_complete.append(np.vstack(target))
    
    return training_complete ,target_complete 

sequence_length = 20
overlap = 15
n_epoch = 5
num = len(list(away['displayName'].value_counts().index))


train_, target_ = get_sequences(a, num, sequence_length, overlap, n_fts=4)

## Training Convolutional Sequential Models and plotting the results

In [ ]:
"""         Batch Size, Seq. Len, Feature length
Input Shape: 8, 20, 54  
Out shape : 8, 2
"""


adagradOpt = Adagrad(lr=0.005, epsilon=1e-08)
model_list = list(away['displayName'].value_counts().index)
batchSize= 8
numOfPrevSteps = 20
featurelen = a.shape[-1]
print('Load models...')
policies2 = []
for _ in model_list:

    conv_model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=32,
                            kernel_size=(5,),
                            activation='relu'),
        tf.keras.layers.Conv1D(filters=32,
                            kernel_size=(8,),
                            activation='relu'),
        tf.keras.layers.Conv1D(filters=32,
                            kernel_size=(9,),
                            activation='relu'),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=2),
    ])
    conv_model.compile(loss='mae', optimizer='adam')
    policies2.append(conv_model)


print('Train Models')
training_loss =[[] for index in range(len(policies2))]
for index in range(len(policies2)):

        inp =train_[index]
        out =target_[index]
        loss = policies2[index].fit(inp, out,epochs =150, verbose=False)
        training_loss[index].append(loss)  
for n,loss in enumerate(training_loss):
    plt.plot(loss[-1].history['loss'],label = str(n))
plt.legend()
plt.show()
print('Finished')

## Plotting the last 18 Trajectory Points of the model

In [ ]:
complete_data = np.squeeze(np.array(a),0)
pred = []
for i in range(38): 
    next_prediction_all = []
    inp_pred_all = []
    n_fts = 4
    if i<20:
        continue
    for index in range(len(policies2)):
        inp = np.array(complete_data[i-sequence_length:i,:])
        next_prediction = policies2[index].predict(np.expand_dims(inp,0))
        next_prediction_all.append(next_prediction.flatten())
        inp_pred_all.append(inp)
    next_prediction_all = np.vstack(next_prediction_all)
    pred.append(next_prediction_all)
    home_players = np.array([complete_data[i+1,index*n_fts:index*n_fts+2] for index in range(len(policies2))])
    plt.scatter(home_players[:,0], home_players[:,1], color='red')
    plt.scatter(next_prediction_all[:,0], next_prediction_all[:,1], color ='b')
    '''
    #input trajectory
    for j in inp_pred_all:
        for k in range(j.shape[0]):
            u = np.array([j[k,index*n_fts:index*n_fts+2] for index in range(len(policies2))])
            plt.scatter(u[:,0], u[:,1], color = 'yellow')
    '''
    plt.show()

## Model Predictions in Black and Targets in Color

In [ ]:
for p in pred:
    plt.scatter(p[:,0], p[:,1],color = 'k')
for t in target_:
    plt.scatter(t[:,0], t[:,1])


## Prediction of ONE single Track 

In [ ]:
next_prediction_all  = []
true_pred = []
for index in range(len(policies2)):

        inp =train_[index]
        out =target_[index]
        next_prediction = policies2[index].predict(inp).flatten()
        next_prediction_all.append(next_prediction)
        true_pred.append(out)
        break
next_prediction_all = np.vstack(next_prediction_all)
true_pred = np.vstack(true_pred)
plt.scatter(next_prediction_all[:,0], next_prediction_all[:,1], color ='b')
plt.scatter(true_pred[:,0], true_pred[:,1], color ='r')

# Here we Train the LSTMs

In [ ]:
a = np.hstack(player_trajectories)
a = np.hstack((a,football_seq))
print(a.shape)
a = a[10:]- np.mean(a, axis=0)
print(a.shape)
a = np.expand_dims(a, axis=0)
train_, target_ = get_sequences(a, num, sequence_length, overlap, n_fts=4)
adagradOpt = Adagrad(lr=0.005, epsilon=1e-08)
model_list = list(away['displayName'].value_counts().index)
batchSize= 8
numOfPrevSteps = 20
featurelen = a.shape[-1]
print('Load models...')
policies2 = []
#### Load the model
for model_name in model_list:
    model = Sequential()
    model.add(tf.keras.layers.Bidirectional(LSTM(512 ,return_sequences=True)))
    model.add(tf.keras.layers.Bidirectional(LSTM(512 , return_sequences=False)))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(2, activation = 'linear') )
    model.compile(loss='mae', optimizer='adam')
    model.reset_states()
    policies2.append(model)   

print('Train Models')
training_loss =[[] for index in range(len(policies2))]
for index in range(len(policies2)):

        inp =train_[index]
        out =target_[index]
        loss = policies2[index].fit(inp, out,epochs =150, verbose=False)
        training_loss[index].append(loss)  
for n,loss in enumerate(training_loss):
    plt.plot(loss[-1].history['loss'],label = str(n))
plt.legend()
plt.show()
print('Finished')

In [ ]:
complete_data = np.squeeze(np.array(a),0)
pred = []
for i in range(38): 
    next_prediction_all = []
    inp_pred_all = []
    n_fts = 4
    if i<20:
        continue
    for index in range(len(policies2)):
        inp = np.array(complete_data[i-sequence_length:i,:])
        next_prediction = policies2[index].predict(np.expand_dims(inp,0))
        next_prediction_all.append(next_prediction.flatten())
        inp_pred_all.append(inp)
    next_prediction_all = np.vstack(next_prediction_all)
    pred.append(next_prediction_all)
    home_players = np.array([complete_data[i+1,index*n_fts:index*n_fts+2] for index in range(len(policies2))])
    plt.scatter(home_players[:,0], home_players[:,1], color='red')
    plt.scatter(next_prediction_all[:,0], next_prediction_all[:,1], color ='b')
    '''
    #input trajectory
    for j in inp_pred_all:
        for k in range(j.shape[0]):
            u = np.array([j[k,index*n_fts:index*n_fts+2] for index in range(len(policies2))])
            plt.scatter(u[:,0], u[:,1], color = 'yellow')
    '''
    plt.show()


### Depricated: We can see that the model seems to collaps to the mean of the labeled targets.

### Solution: Substract the mean of the player positions to get good predictions. -> 

In [ ]:
for p in pred:
    plt.scatter(p[:,0], p[:,1],color = 'k')
for t in target_:
    plt.scatter(t[:,0], t[:,1])


In [ ]:
next_prediction_all  = []
true_pred = []
for index in range(len(policies2)):

        inp =train_[index]
        out =target_[index]
        next_prediction = policies2[index].predict(inp)
        next_prediction_all.append(next_prediction)
        true_pred.append(out)
        break
next_prediction_all = np.vstack(next_prediction_all)
true_pred = np.vstack(true_pred)
plt.scatter(next_prediction_all[:,0], next_prediction_all[:,1], color ='b')
plt.scatter(true_pred[:,0], true_pred[:,1], color ='r')

# Whats next?
* Stabilize training
* Change data prep, so that mean does not influence position
* Improve plotting

# Stabilizing the training

After ensuring that the training is taking place for a single sample, Lets train on a week data. 
We could further implement **DAgger**.

## Load the data again and Sort the players
For input of the data, I sorted the players. First the defensive linup, then the offense and lastly the football


In [ ]:
data =  pd.read_csv(weeks)
playdf = pd.read_csv(plays)

gameIndex = data['gameId'].value_counts().index
defensive_lineup = ['CB','SS', 'FS', 'S', 'MLB', 'OLB', 'ILB', 'LB', 'DB', 'DE']
offensive_lineup = ['WR', 'RB', 'HB','TE', 'FB', 'QB']
sorter = defensive_lineup + offensive_lineup + ['NaN'] #nan is football
sorterIndex = dict(zip(sorter, range(len(sorter))))
sequences = []
seq_play = []

In [ ]:
for gameId in gameIndex:
    print('Starting with game: ' + str(gameId))
    start_game = time.time()
    playIds = data.query('gameId== ' + str(gameId))['playId'].value_counts().index
    for playId in playIds:
        play = data.query('playId == ' +str(playId) + 'and gameId== ' + str(gameId))
        yardline = playdf.query('playId == ' +str(playId) + 'and gameId== ' + str(gameId))
        start = time.time()
        if (set(list(play.position.value_counts().index)) <= set(sorter)) and (len(play.displayName.value_counts().index)==17): 
            try:
                play = play.apply(compute_speed, axis=1)
                player_trajectories = []
                football = play.query('team=="football"')
                football = football.sort_values('frameId')
                devensive_players = 0
                off_player = 0 
                for player in sorter:
                    trajectory1 = play.loc[play['position']==player]
                    for i in trajectory1.displayName.value_counts().index:
                        if i == 'Football':
                            continue
                        if player in defensive_lineup:
                            devensive_players +=1    
                        else:
                            off_player +=1 
                        trajectory = trajectory1.loc[trajectory1['displayName']==i]
                        player_seq = trajectory.sort_values(['frameId'])
                        try:
                            dist_to_ball_x = np.absolute(player_seq[['x']].to_numpy()-football[['x']].to_numpy())
                            dist_to_ball_y = np.absolute(player_seq[['y']].to_numpy()-football[['y']].to_numpy())
                        except:
                            px = np.vstack([player_seq[['x']].to_numpy(),player_seq[['x']].to_numpy()[-1]])
                            py = np.vstack([player_seq[['y']].to_numpy(),player_seq[['y']].to_numpy()[-1]])
                            dist_to_ball_x = np.absolute(px-football[['x']].to_numpy())
                            dist_to_ball_y = np.absolute(py-football[['y']].to_numpy())
                        player_seq['x'] -=  football['x'].iloc[0]
                        player_seq['y'] -=  football['y'].iloc[0]
                        player_seq = player_seq[['x','y','v_x','v_y']].to_numpy()
                        final_player_traj = np.hstack((player_seq, dist_to_ball_x, dist_to_ball_y))
                        player_trajectories.append(final_player_traj)
                football['x'] -=  football['x'].iloc[0]
                football['y'] -=  football['y'].iloc[0]
                football_seq = football[['x','y']].to_numpy()
                a = np.hstack(player_trajectories)
                a = np.hstack((a,football_seq))
                print('defensive players: ' + str(devensive_players))
                print('offensive players: ' + str(off_player))
                sequences.append(a)
                seq_play.append(yardline)
            except:
                print('Play wasnt added to sequences: ' + str(playId))
        else:
            print('Abort')
            print(play.position.value_counts().index)
            print(len(play.displayName.value_counts().index))
            continue
    print('Done with one game')
    print(time.time()-start_game)


In [ ]:
### Preprocessing Alterations 
num = len(defensive_lineup)
sequence_length = 15
overlap = 6
train_, target_ = get_sequences(sequences, num, sequence_length, overlap, n_fts=6)

In [ ]:
train_datasets = [tf.data.Dataset.from_tensor_slices((train, target)) for train, target in zip(train_, target_)]

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 100
for index in range(len(train_datasets)):    
    train_datasets[index] = train_datasets[index].shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
model_list = defensive_lineup
batchSize= 8
numOfPrevSteps = 20
featurelen = a.shape[-1]
print('Load models...')
policies2 = []
#### Load the model
for model_name in model_list:
    model = Sequential()
    model.add(tf.keras.layers.Bidirectional(LSTM(512 ,return_sequences=True)))
    model.add(tf.keras.layers.Bidirectional(LSTM(512 , return_sequences=False)))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(2, activation = 'linear') )
    model.compile(loss='mae', optimizer='adam')
    #model.load_weights(model_name)
    #model.load_weights('init_weights_minibatch1024_10epochs.h5') # Load the pretrained model
    model.reset_states()
    policies2.append(model)   



In [ ]:
print('Train Models')
training_loss =[[] for index in range(len(policies2))]
for index,(model, trainset) in enumerate(zip(policies2, train_datasets)):
    print('Starting with Model ' + str(index))
    loss = model.fit(trainset ,epochs =50)
    training_loss[index].append(loss)  
for n,loss in enumerate(training_loss):
    plt.plot(loss[-1].history['loss'],label = str(n))
plt.legend()
plt.show()
print('Finished')

In [ ]:
def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=50,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
                 color='yellow')
    return fig, ax


## Do sample predictions of a sequence

In [ ]:
pred = []
complete_data = np.array(sequences[0])
for i in range(sequences[0].shape[0]):
    next_prediction_all = []
    inp_pred_all = []
    n_fts = 6
    if i<15:
        continue
    for index in range(len(policies2)):
        inp = np.array(complete_data[i-sequence_length:i,:])
        next_prediction = policies2[index].predict(np.expand_dims(inp,0))
        next_prediction_all.append(next_prediction.flatten())
    next_prediction_all = np.vstack(next_prediction_all)
    pred.append(next_prediction_all)
    plt.scatter(next_prediction_all[:,0], next_prediction_all[:,1], color='orange', s=30)

In [ ]:
fig, ax = create_football_field()
play = data.query('gameId == 2018100703 and playId == 4943 ')
example_play_home = play.query('team=="home"')
example_play_away = play.query('team == "away"')
example_football = play.query('team == "football"')
example_play_home.plot(x='x', y='y', kind='scatter', ax=ax, color='orange', s=30, legend='Away')
#example_play_away.plot(x='x', y='y', kind='scatter', ax=ax, color='blue', s=30, legend='Home')
example_football.plot(x='x', y='y', kind='scatter', ax=ax, color='red', s=30, legend='football')

example_play_home = play.query('team=="home"')
for p in pred: 
    offset = [example_football.query('frameId ==1')['x'].to_numpy(),  example_football.query('frameId ==1')['y'].to_numpy()]
    ax.scatter(p[:,0].flatten() + offset[0],
               p[:,1].flatten() +offset[1], color='blue', s=30)

plt.legend()
plt.show()